<h2>Applied DS Capstone Project</h2>
<p>This notebook is the Capstone Project for the Applied Data Science Certification path on Coursera.org</p>

In [1]:
import pandas as pd
import numpy as np
print('hello capstone project course!'.title())

Hello Capstone Project Course!


Replicate the exploration and clustering of New York City using Toronto

In [2]:
t = pd.read_html('https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Toronto#Table')

In [74]:
provided = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

In [76]:
for i, tbl in enumerate(provided):
    if "Borough" in tbl.columns:
        print(f'Your table is at index {i}')
        break
provided[i]

Your table is at index 0


,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [79]:
df_postal = provided[0][provided[0]['Borough']!='Not assigned'].reset_index(drop = True)
df_postal

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [82]:
print(df_postal['Borough'].unique())

['North York' 'Downtown Toronto' 'Etobicoke' 'Scarborough' 'East York'
 'York' 'East Toronto' 'West Toronto' 'Central Toronto' 'Mississauga'] ['East York' 'Etobicoke' 'North York' 'Old City of Toronto' 'Scarborough'
 'York']


In [3]:
len(t)

17

In [4]:
for i, tbl in enumerate(t):
    if "CDN number" in tbl.columns:
        print(f'Your table is at index {i}')
        break
t[i]        

Your table is at index 11


,CDN number,City-designated neighbourhood,Former city/borough,Neighbourhoods covered,Map,Unnamed: 5
0,129,Agincourt North,Scarborough,Agincourt and Brimwood,NaN,NaN
1,128,Agincourt South-Malvern West,Scarborough,Agincourt and Malvern,NaN,NaN
2,20,Alderwood,Etobicoke,Alderwood,NaN,NaN
3,95,Annex,Old City of Toronto,The Annex and Seaton Village,NaN,NaN
4,42,Banbury-Don Mills,North York,Don Mills,NaN,NaN
...,...,...,...,...,...,...
135,94,Wychwood,Old City of Toronto,NaN,NaN,NaN
136,100,Yonge and Eglinton,Old City of Toronto,Chaplin Estates,NaN,NaN
137,97,Yonge-St.Clair,Old City of Toronto,NaN,NaN,NaN
138,27,York University Heights,North York,NaN,NaN,NaN


In [7]:
toronto_raw = t[11].loc[:,['Former city/borough', 'City-designated neighbourhood']]
toronto_raw.columns = ['Borough', 'Neighborhood']

In [18]:
toronto_raw.sort_values(['Borough', 'Neighborhood'], inplace=True)

In [20]:
toronto_raw.reset_index(drop=True, inplace=True)
toronto_raw

,Borough,Neighborhood
0,East York,Broadview North
1,East York,Crescent Town
2,East York,Danforth - East York
3,East York,Leaside-Bennington
4,East York,O'Connor-Parkview
...,...,...
135,York,Lambton Baby Point
136,York,Mount Dennis
137,York,Oakwood Village
138,York,Rockcliffe-Smythe


In [21]:
import json

from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans
from pandas.io.json import json_normalize

import folium as fm
import matplotlib.cm as cm
import matplotlib.colors as colors

In [63]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
t_lat = location.latitude
t_long = location.longitude
print(f'The geograpical coordinates of {address} are {t_lat:10f}, {t_long:10f}.')

The geograpical coordinates of Toronto, ON are  43.653482, -79.383935.


In [34]:
for rough in toronto_raw['Borough'].unique():
    address = rough +', Toronto, ON'

    geolocator = Nominatim(user_agent="toronto_explorer")
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    print(f'The geograpical coordinates of {rough:>19} are {latitude:10f}, {longitude:10f}.')

The geograpical coordinates of           East York are  43.699971, -79.332520.
The geograpical coordinates of           Etobicoke are  43.643556, -79.565633.
The geograpical coordinates of          North York are  43.754326, -79.449117.
The geograpical coordinates of Old City of Toronto are  43.620412, -79.373492.
The geograpical coordinates of         Scarborough are  43.773077, -79.257774.
The geograpical coordinates of                York are  43.689619, -79.479188.


In [26]:
toronto_raw['Borough'].unique()

array(['East York', 'Etobicoke', 'North York', 'Old City of Toronto',
       'Scarborough', 'York'], dtype=object)

In [37]:
borough_filter = toronto_raw['Borough'] == 'East York'
one_borough = toronto_raw[borough_filter]

In [38]:
for hood in one_borough['Neighborhood']:
    address = hood +', Toronto, ON'
    
    try:
        geolocator = Nominatim(user_agent="toronto_explorer")
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        print(f'The geograpical coordinates of {hood:>19} are {latitude:10f}, {longitude:10f}.')
    except:
        print(f'{hood} was not mapped')

The geograpical coordinates of     Broadview North are  43.683924, -79.356964.
The geograpical coordinates of       Crescent Town are  43.695403, -79.293099.
The geograpical coordinates of Danforth - East York are  43.686433, -79.300355.
Leaside-Bennington was not mapped
The geograpical coordinates of   O'Connor-Parkview are  43.707842, -79.311074.
The geograpical coordinates of       Old East York are  43.699971, -79.332520.
The geograpical coordinates of    Thorncliffe Park are  43.704553, -79.345407.
Woodbine-Lumsden was not mapped


In [ ]:
latlong_dict = {'Lat': [], 'Long':[]}

for hood in toronto_raw['Neighborhood']:
    address = hood +', Toronto, ON'
    
    try:
        geolocator = Nominatim(user_agent="toronto_explorer")
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        latlong_dict['Lat'].append(latitude)
        latlong_dict['Long'].append(longitude)

    except:
        latlong_dict['Lat'].append(0)
        latlong_dict['Long'].append(0)
        
latlong_dict

In [48]:
toronto = toronto_raw
toronto['Lat'] = pd.DataFrame(latlong_dict['Lat'])
toronto['Long'] = pd.DataFrame(latlong_dict['Long'])

In [53]:
print(len(toronto[toronto['Lat']==0]), len(toronto[toronto['Lat']!=0]))

33 107


In [54]:
t = toronto

In [60]:
t.drop(t[t['Lat']==0].index, inplace=True)

In [61]:
t

,Borough,Neighborhood,Lat,Long
0,East York,Broadview North,43.683924,-79.356964
1,East York,Crescent Town,43.695403,-79.293099
2,East York,Danforth - East York,43.686433,-79.300355
4,East York,O'Connor-Parkview,43.707842,-79.311074
5,East York,Old East York,43.699971,-79.332520
...,...,...,...,...
132,York,Caledonia-Fairbank,43.686753,-79.459575
134,York,Keelesdale-Eglinton West,43.690158,-79.474998
136,York,Mount Dennis,43.686960,-79.489551
137,York,Oakwood Village,43.682725,-79.438055


In [70]:
t_map = fm.Map(location = [t_lat, t_long], zoom_start=10)

for row in t.iterrows():
    hood = row[1] #borough = [0], hood = [1], lat = [2], long = [3]
    
    label = f'{hood[1]}, {hood[0]}' #hood, borough
    fm.CircleMarker(
        [hood[2], hood[3]], #LAT, LONG
        radius = 5,
        popup = label,
        color = 'blue',
        fill=True,
        fill_color = '#3186cc',
        fill_opacity = .7).add_to(t_map)
    
t_map

In [73]:
toronto_raw.to_csv('Toronto_raw.csv')